# Packaging software in images

You can create a Docker image by either modifying an existing image inside a
container or defining and executing a build script called a Dockerfile.

## Building Docker images from a container

It’s easy to get started building images if you’re already familiar with using containers.
Remember, a union filesystem (UFS) mount provides a container’s filesystem. Any changes
that you make to the filesystem inside a container will be written as new layers owned
by the container that created them.

Before you work with real software, the next section details the typical workflow
with a “Hello, World” example.

### Packaging “Hello, World”

The basic workflow for building an image from a container includes three steps. 

1. First, you create a container from an existing image. You choose the image based on what
you want to be included with the new finished image and the tools you need to make
the changes.
2. The second step is to modify the filesystem of the container. These changes will be
written to a new layer of the container’s union filesystem. We’ll revisit the relationship
between images, layers, and repositories later in this chapter.

3. Once the changes have been made, the last step is to commit those changes. Then
you’ll be able to create new containers from the resulting image. 

With these steps in mind, work through the following commands to create a new image
named hw_image:

Modifies file in container:

    docker container run --name hw_container \
            ubuntu:latest \
            touch /HelloWorld


Commits change to new image:

    docker image ls
    docker container commit hw_container hw_image
    docker image ls

Removes changed container

    docker container rm -vf hw_container

Examines file in new container

    docker container run --rm \
        hw_image \
        ls -l /HelloWorld

If that seems stunningly simple, you should know that it does become a bit more
nuanced as the images you produce become more sophisticated, but the basic steps
will always be the same. Now that you have an idea of the workflow, you should try to
build a new image with real software. In this case, you’ll be packaging a program
called Git.

### Preparing packaging for Git

## Building images automatically with Dockerfiles

A Dockerfile is a text file that contains instructions for building an image. The
Docker image builder executes the Dockerfile from top to bottom, and the instructions
can configure or change anything about an image. Building images from
Dockerfiles makes tasks like adding files to a container from your computer simple
one-line instructions. Dockerfiles are the most common way to describe how to
build a Docker image.

Once an
image’s build is defined in code, it is simple to track changes in version control, share
with team members, optimize, and secure.

### Packaging Git with a Dockerfile

You
should recognize many of the details and advantages of working with a Dockerfile as
we translate the image build process from manual operations to code.
First, create a new directory, and from that directory create a new file with your
favorite text editor. Name the new file Dockerfile. Write the following five lines and
then save the file:

> Ustvarimo datoteko Dockerfile

    # An example Dockerfile for installing Git on Ubuntu
    FROM ubuntu:latest
    LABEL maintainer="leon@ltfe.org"
    RUN apt-get update && apt-get install -y git
    ENTRYPOINT ["git"]

Before dissecting this example, build a new image from it with the docker image
build command from the same directory containing the Dockerfile and tag the
image with auto:

    docker image build --tag ubuntu-git:auto .

This outputs several lines about steps and output from apt-get, and will finally display
a message like this:

    Successfully built cc63aeb7a5a2
    Successfully tagged ubuntu-git:auto

Running this command starts the build process. When it’s completed, you should
have a brand-new image that you can test. View the list of all your ubuntu-git images
and test the newest one with this command:

    docker image ls

Now you can run a Git command using the new image:

    docker container run --rm ubuntu-git:auto

These commands demonstrate that the image you built with the Dockerfile works and
is functionally equivalent to the one you built by hand. Examine what you did to
accomplish this:

First, you created a Dockerfile with four instructions:
- `FROM ubuntu:latest` — Tells Docker to start from the latest Ubuntu image just as
you did when creating the image manually.
-  `LABEL maintainer` — Sets the maintainer name and email for the image. Providing
this information helps people know whom to contact if there’s a problem
with the image. This was accomplished earlier when you invoked commit.
- `RUN apt-get update && apt-get install -y git` — Tells the builder to run the
provided commands to install Git.
- `ENTRYPOINT ["git"]` — Sets the entrypoint for the image to git.

Dockerfiles, like most scripts, can include comments. Any line beginning with a # will
be ignored by the builder.

It’s important for Dockerfiles of any complexity to be welldocumented.
In addition to improving Dockerfile maintainability, comments help
people audit images that they’re considering for adoption and spread best practices.

The only special rule about Dockerfiles is that the first instruction must be FROM. If
you’re starting from an empty image and your software has no dependencies, or you’ll
provide all the dependencies, then you can start from a special empty repository
named scratch.

After you saved the Dockerfile, you started the build process by invoking the docker
image build command. The command had one flag set and one argument. The --tag
flag (or -t for short) specifies the full repository designation that you want to use for the
resulting image. In this case, you used ubuntu-git:auto. The argument that you
included at the end was a single period. That argument told the builder the location of
the Dockerfile. The period told it to look for the file in the current directory.

The docker image build command has another flag, --file (or -f for short), that
lets you set the name of the Dockerfile. Dockerfile is the default, but with this flag
you could tell the builder to look for a file named BuildScript or release-image.df.
This flag sets only the name of the file, not the location. That must always be specified
in the location argument.

The builder works by automating the same tasks that you’d use to create images by
hand. Each instruction triggers the creation of a new container with the specified
modification. After the modification has been made, the builder commits the layer
and moves on to the next instruction and container created from the fresh layer.

The builder validated that the image specified by the FROM instruction was installed
as the first step of the build. If it were not, Docker would have automatically tried to
pull the image.

Installing software packages is one of the most common use cases
for the RUN instruction. You should explicitly install each software package needed by
your container to ensure that it is available when needed.

If you’re not interested in reams of build process output, you can invoke the
docker image build command with the --quiet or -q flag. Running in quiet mode
will suppress all output from the build process and management of intermediate containers.
The only output of the build process in quiet mode is the resulting image ID,
which looks like this:

A new layer is being added to the resulting image after each step in the build.
Although this means you could potentially branch on any of these steps, the more
important implication is that the builder can aggressively cache the results of each
step. If a problem with the build script occurs after several other steps, the builder can
restart from the same position after the problem has been fixed.

Add this line to the end of your Dockerfile:
    
    RUN This will not work

Then run the build again:
    
    docker image build --tag ubuntu-git:auto .

Steps 1 through 4 were skipped because they were already built during your last build.
Step 5 failed because there’s no program with the name This in the container. The
container output was valuable in this case because the error message informs you
about the specific problem with the Dockerfile. If you fix the problem, the same steps
will be skipped again, and the build will succeed, resulting in output like Successfully
built d7a8ee0cebd4.

The use of caching during the build can save time if the build includes downloading
material, compiling programs, or anything else that is time-intensive. If you need a
full rebuild, you can use the --no-cache flag on docker image build to disable the
use of the cache. Make sure you’re disabling the cache only when required because it
will place much more strain on upstream source systems and image-building systems.

### A Dockerfile primer

Dockerfiles are expressive and easy to understand because of their terse syntax that
allows for comments. You can keep track of changes to Dockerfiles with any versioncontrol
system. Maintaining multiple versions of an image is as simple as maintaining multiple Dockerfiles.

The Dockerfile build process itself uses extensive caching to aid
rapid development and iteration. The builds are traceable and reproducible. They
integrate easily with existing build systems and many continuous integration tools.
With all these reasons to prefer Dockerfile builds to handmade images, it’s important
to learn how to write them.

#### Metadata instructions

The first example builds a base image and two other images with distinct versions of
the mailer program you used in chapter 2. The purpose of the program is to listen for
messages on a TCP port and then send those messages to their intended recipients.
The first version of the mailer will listen for messages but only log those messages. The
second will send the message as an HTTP POST to the defined URL.

One of the best reasons to use Dockerfile builds is that they simplify copying files
from your computer into an image. But it’s not always appropriate for certain files to
be copied to images. The first thing to do when starting a new project is to define
which files should never be copied into any images. You can do this in a file called
.dockerignore. In this example, you’ll create three Dockerfiles, and none needs to be
copied into the resulting images.

Use your favorite text editor to create a new file named .dockerignore and copy in
the following lines:
    
    .dockerignore
    mailer-base.df
    mailer-logging.df
    mailer-live.df

Save and close the file when you’re finished. This will prevent the .dockerignore file,
or files named mailer-base.df, mailer-logging.df, or mailer-live.df, from ever being
copied into an image during a build. With that bit of accounting finished, you can
begin working on the base image.

Building a base image helps create common layers. Each version of the mailer will
be built on top of an image called mailer-base. When you create a Dockerfile, you
need to keep in mind that each Dockerfile instruction will result in a new layer being
created. Instructions should be combined whenever possible because the builder
won’t perform any optimization. Putting this in practice, create a new file named
mailer-base.df and add the following lines:

```Dockerfile    
FROM debian:buster-20190910
LABEL maintainer="dia@allingeek.com"
RUN groupadd -r -g 2200 example && \
    useradd -rM -g example -u 2200 example
ENV APPROOT="/app" \
    APP="mailer.sh" \
    VERSION="0.6"
LABEL base.name="Mailer Archetype" \
    base.version="${VERSION}"
WORKDIR $APPROOT
ADD . $APPROOT
ENTRYPOINT ["/app/mailer.sh"]
EXPOSE 33333
# Do not set the default user in the base otherwise
# implementations will not be able to update the image
# USER example:example
```

Put it all together by running the docker image build command from the directory
where the mailer-base file is located. The -f flag tells the builder which filename to
use as input:

    docker image build -t dockerinaction/mailer-base:0.6 -f mailer-base.df .

Five new instructions are introduced in this Dockerfile. The first new instruction is
ENV. ENV sets environment variables for an image, similar to the --env flag on docker
container run or docker container create. In this case, a single ENV instruction is
used to set three distinct environment variables. That could have been accomplished
with three subsequent ENV instructions, though doing so would result in the creation
of three layers. You can keep instructions easy to read by using a backslash to escape
the newline character (just as in shell scripting).

Environment variables declared in the Dockerfile are made available to the resulting
image but can be used in other Dockerfile instructions as substitutions. In this Dockerfile,
the environment variable VERSION was used as a substitution in the next new
instruction, LABEL.

The LABEL instruction is used to define key/value pairs that are recorded as additional
metadata for an image or container. This mirrors the --label flag on docker run and
docker create. Like the ENV instruction before it, multiple labels can and should be
set with a single instruction. In this case, the value of the VERSION environment variable
was substituted for the value of the base.version label. By using an environment
variable in this way, the value of VERSION will be available to processes running inside a
container as well as recorded to an appropriate label. This increases maintainability of
the Dockerfile because it’s more difficult to make inconsistent changes when the valueis
set in a single location.

The next two instructions are WORKDIR and EXPOSE. These are similar in operation to
their corresponding flags on the docker run and docker create commands. An environment
variable was substituted for the argument to the WORKDIR command.

The result of the WORKDIR instruction will be an image with the default working directory
set to /app. Setting WORKDIR to a location that doesn’t exist will create that location
just as it would with the command-line option. Last, the EXPOSE command creates
a layer that opens TCP port 33333.

The parts of this Dockerfile that you should recognize are the FROM, LABEL, and
ENTRYPOINT instructions. In brief, the FROM instruction sets the layer stack to start from
the debian:buster-20190910 image. Any new layers built will be placed on top of that
image. The LABEL instruction adds key/value pairs to the image’s metadata. The
ENTRYPOINT instruction sets the executable to run at container startup. Here, it’s setting
the instruction to exec ./mailer.sh and using the shell form of the instruction.

The ENTRYPOINT instruction has two forms: the shell form and an exec form. The
shell form looks like a shell command with whitespace-delimited arguments. The exec
form is a string array in which the first value is the command to execute and the
remaining values are arguments. A command specified using the shell form would be
executed as an argument to the default shell. Specifically, the command used in this
Dockerfile will be executed as /bin/sh –c 'exec ./mailer.sh' at runtime. Most
importantly, if the shell form is used for ENTRYPOINT, all other arguments provided by
the CMD instruction or at runtime as extra arguments to docker container run will be
ignored. This makes the shell form of ENTRYPOINT less flexible.

You can see from the build output that the ENV and LABEL instructions each resulted
in a single step and layer. But the output doesn’t show that the environment variable values
were substituted correctly. To verify that, you’ll need to inspect the image:

    docker inspect dockerinaction/mailer-base:0.6

> Remember, the docker inspect command can be used to view the metadata
of either a container or an image. In this case, you used it to inspect an
image.

The metadata makes it clear that the environment variable substitution works. You can
use this form of substitution in the ENV, ADD, COPY, LABEL, WORKDIR, VOLUME, EXPOSE,
and USER instructions.

The last commented line is a metadata instruction, USER. It sets the user and group
for all further build steps and containers created from the image. In this case, setting
it in a base image would prevent any downstream Dockerfiles from installing software.
That would mean that those Dockerfiles would need to flip the default back and forth
for permission. Doing so would create at least two additional layers. The better
approach would be to set up the user and group accounts in the base image and let
the implementations set the default user when they’ve finished building.

The most curious thing about this Dockerfile is that the ENTRYPOINT is set to a file
that doesn’t exist. The entrypoint will fail when you try to run a container from this
base image. But now that the entrypoint is set in the base image, that’s one less layer
that will need to be duplicated for specific implementations of the mailer. The next
two Dockerfiles build different mailer.sh implementations.

#### Filesystem instructions

## Best practices for writing Dockerfiles

In [ ]:
https://docs.docker.com/develop/develop-images/dockerfile_best-practices/
    